In [1]:
import cv2
import mediapipe as mp
import threading
import time
import numpy as np
import ctypes
import webbrowser
import subprocess

sending = False  # global flag


# =========================
# SCREEN SIZE (Windows)
# =========================
def get_screen_size():
    user32 = ctypes.windll.user32
    W = user32.GetSystemMetrics(0)
    H = user32.GetSystemMetrics(1)
    return W, H


# =========================
# 1) FULLSCREEN HUAWEI BUBBLE WINDOW
# =========================
def draw_huawei_ring(frame, angle):
    h, w = frame.shape[:2]
    center = (w // 2, h // 2)

    overlay = frame.copy()
    cv2.rectangle(overlay, (0, 0), (w, h), (0, 0, 0), -1)
    frame = cv2.addWeighted(overlay, 0.45, frame, 0.55, 0)

    outer = frame.copy()
    cv2.circle(outer, center, 220, (255, 255, 255), 22)
    outer = cv2.GaussianBlur(outer, (61, 61), 0)
    frame = cv2.addWeighted(outer, 0.25, frame, 0.75, 0)

    cv2.circle(frame, center, 160, (255, 255, 255), 18)
    cv2.ellipse(frame, center, (160, 160), 0, angle, angle + 60, (230, 230, 230), 20)
    return frame


def bubble_loop():
    global sending
    win = "Huawei Bubble"
    cv2.namedWindow(win, cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty(win, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    W, H = get_screen_size()
    bubble = np.zeros((H, W, 3), dtype=np.uint8)

    angle = 0
    while sending:
        frame = bubble.copy()
        frame = draw_huawei_ring(frame, angle)
        cv2.imshow(win, frame)

        angle = (angle + 6) % 360

        if cv2.waitKey(1) & 0xFF == 27:
            sending = False
            break

        time.sleep(0.01)

    cv2.destroyWindow(win)


# =========================
# 2) PC AUTOMATION ACTIONS
# =========================
def run_action(action_key):
    # Web
    if action_key == "youtube":
        webbrowser.open("https://www.youtube.com")
    elif action_key == "duolingo":
        webbrowser.open("https://www.duolingo.com")
    elif action_key == "gmail":
        webbrowser.open("https://mail.google.com")
    elif action_key == "linkedin":
        webbrowser.open("https://www.linkedin.com")

    # Apps
    elif action_key == "notepad":
        subprocess.Popen("notepad.exe", shell=True)
    elif action_key == "calculator":
        subprocess.Popen("calc.exe", shell=True)
    elif action_key == "cmd":
        subprocess.Popen("cmd.exe", shell=True)

    # System
    elif action_key == "explorer":
        subprocess.Popen("explorer.exe", shell=True)
    elif action_key == "settings":
        subprocess.Popen("start ms-settings:", shell=True)

    else:
        print("Unknown action:", action_key)


def execute_action(action_key):
    global sending
    start_time = time.time()
    MIN_BUBBLE_SECONDS = 1.5

    try:
        print(f"✅ Running action: {action_key}")
        run_action(action_key)
    except Exception as e:
        print("❌ Action failed:", e)

    elapsed = time.time() - start_time
    if elapsed < MIN_BUBBLE_SECONDS:
        time.sleep(MIN_BUBBLE_SECONDS - elapsed)

    sending = False


# =========================
# 3) MEDIAPIPE + CAMERA WINDOW (SMALL + MENU SYSTEM + HOLD)
# =========================
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

cap = cv2.VideoCapture(0)

# Small camera window
win_cam = "Gesture Detection"
cv2.namedWindow(win_cam, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow(win_cam, 360, 240)
cv2.moveWindow(win_cam, 30, 30)

# ========= MENU DATA =========
mode = "web"

MENU = {
    "web": ["youtube", "duolingo", "gmail", "linkedin"],
    "apps": ["notepad", "calculator", "cmd"],
    "system": ["explorer", "settings"]
}

selected_index = 0
selected_action = MENU[mode][selected_index]

# Threads
bubble_thread = None
action_thread = None

# Stability + Cooldown
stable_count_needed = 8
cooldown_sec = 2.0
last_trigger_time = 0
last_seen = None
stable_count = 0

# ✅ HOLD TO CONFIRM
hold_start_time = None
HOLD_SECONDS = 1.2  # gesture ko itna seconds hold karna hoga


def compute_fingers(results, hand_landmarks, i):
    landmarks = hand_landmarks.landmark
    fingers = []
    hand_label = results.multi_handedness[i].classification[0].label

    # Thumb
    if hand_label == "Right":
        fingers.append(1 if landmarks[4].x < landmarks[3].x else 0)
    else:
        fingers.append(1 if landmarks[4].x > landmarks[3].x else 0)

    # Other fingers
    tips = [8, 12, 16, 20]
    for tip in tips:
        fingers.append(1 if landmarks[tip].y < landmarks[tip - 2].y else 0)

    return sum(fingers), hand_label


def set_mode(new_mode):
    global mode, selected_index, selected_action
    mode = new_mode
    selected_index = 0
    selected_action = MENU[mode][selected_index]


def next_item():
    global selected_index, selected_action
    selected_index = (selected_index + 1) % len(MENU[mode])
    selected_action = MENU[mode][selected_index]


while True:
    success, img = cap.read()
    if not success:
        break

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    total_fingers = None
    hand_label = ""

    if results.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            total_fingers, hand_label = compute_fingers(results, hand_landmarks, i)
            break

    # ===== Stability logic =====
    if total_fingers is None:
        last_seen = None
        stable_count = 0
        hold_start_time = None
    else:
        if total_fingers == last_seen:
            stable_count += 1
        else:
            last_seen = total_fingers
            stable_count = 1
            hold_start_time = None  # ✅ gesture change => hold reset

    # ===== UI =====
    if total_fingers is not None:
        cv2.putText(img, f'{hand_label} Fingers: {total_fingers}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(img, f'Stable: {stable_count}/{stable_count_needed}', (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    else:
        cv2.putText(img, 'No hand detected', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    # Menu UI
    cv2.putText(img, f"Mode: {mode.upper()}", (10, 95),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
    cv2.putText(img, f"Selected: {selected_action}", (10, 120),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
    cv2.putText(img, "2=WEB  3=APPS  4=SYSTEM  1=NEXT  5=RUN  0=CANCEL", (10, img.shape[0]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)

    now = time.time()
    can_trigger = (now - last_trigger_time) > cooldown_sec and (not sending)

    # ✅ Stable + Hold logic
    if total_fingers is not None and stable_count >= stable_count_needed and can_trigger:
        if hold_start_time is None:
            hold_start_time = time.time()

        hold_elapsed = time.time() - hold_start_time

        # show hold progress
        cv2.putText(img, f"HOLD: {hold_elapsed:.1f}/{HOLD_SECONDS}s",
                    (10, 150),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (0, 255, 255),
                    2)

        if hold_elapsed >= HOLD_SECONDS:
            hold_start_time = None
            last_trigger_time = time.time()

            # Mode select
            if total_fingers == 2:
                set_mode("web")
                print("Mode set: WEB")

            elif total_fingers == 3:
                set_mode("apps")
                print("Mode set: APPS")

            elif total_fingers == 4:
                set_mode("system")
                print("Mode set: SYSTEM")

            # Next item
            elif total_fingers == 1:
                next_item()
                print("Next:", selected_action)

            # Run
            elif total_fingers == 5:
                sending = True

                bubble_thread = threading.Thread(target=bubble_loop, daemon=True)
                bubble_thread.start()

                action_thread = threading.Thread(target=execute_action, args=(selected_action,), daemon=True)
                action_thread.start()

            # Cancel
            elif total_fingers == 0:
                print("Cancelled")

            stable_count = 0
            last_seen = None

    cv2.imshow(win_cam, img)

    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\prach\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Mode set: WEB
Next: duolingo
✅ Running action: duolingo
Next: gmail
✅ Running action: gmail
Mode set: APPS
✅ Running action: notepad
Next: calculator
✅ Running action: calculator
Next: cmd
✅ Running action: cmd
Mode set: SYSTEM
✅ Running action: explorer
Next: settings
Mode set: SYSTEM
Mode set: SYSTEM
Mode set: SYSTEM
✅ Running action: explorer
✅ Running action: explorer
Next: settings
✅ Running action: settings
Next: explorer
Next: settings
Next: explorer
Next: settings
Next: explorer
Next: settings
✅ Running action: settings


KeyboardInterrupt: 